In [33]:
include("markov-drum-utilities.jl")  #loads AugMatrix, AugDblMatrix, insert, functionName, abptest, makeppheader

makeppheader (generic function with 1 method)

In [34]:
#input a vector corresponding to a probability distribution and output an index according to that distribution
function markovrand(nextv)
    if round(sum(nextv), digits = 6) != 1.0
        error(nextv, " is not a stochastic vector")
    end
    maxstates = length(nextv) - 1
    r = rand(Float64)
    current = maxstates
    for i = 0:maxstates
        if r <= nextv[i + 1]
            current = i
            break
        else
            r = r - nextv[i + 1]
        end
    end
    return(current)
end


markovrand (generic function with 1 method)

In [35]:
#output bass notes in 2*$beats notes
function ghostbassgen(beats::Int, trans::AugMatrix)
    transmat = trans.mat
    iv = trans.init
    nextv = iv
    L = []
    for j = 1:beats
        current = markovrand(nextv)
#print(current, " ")
        bincurrent = digits(current, base = 3, pad = 2)
#print(bincurrent, "\n")
        append!(L, [bincurrent[2], bincurrent[1]])
        nextv = transmat[:, current + 1]
    end
    return(L)
end

ghostbassgen (generic function with 1 method)

In [36]:
#0 = rest, 1 = bass, 2 = LH
                #00    #01  #02   #10  #11  #12  #zeros
rockBassMatrix = [0.1e0 1//6  0   1//6  0.5    0 0 0 0 #00
                    0   1//3  0     0    0    0  0 0 0  #01
                    0   1//6  0.1   1//6  0.5   0 0 0 0  #02
                    0.45 1//3  0.45  1//3 0    0 0 0 0  #10
                    0.45 0    0.45  1//3  0    0 0 0 0  #11
                    0.0  0    0      0   0    0 0 0 0    #12
                    zeros(3,9)]                 #zeros

rockBassInitial = [0.1,0, 0.1, 0.4, 0.4, 0, 0, 0, 0]

rockBass = AugMatrix(rockBassMatrix, rockBassInitial)

9×9 Matrix{Float64}:
 0.1   0.166667  0.0   0.166667  0.5  0.0  0.0  0.0  0.0
 0.0   0.333333  0.0   0.0       0.0  0.0  0.0  0.0  0.0
 0.0   0.166667  0.1   0.166667  0.5  0.0  0.0  0.0  0.0
 0.45  0.333333  0.45  0.333333  0.0  0.0  0.0  0.0  0.0
 0.45  0.0       0.45  0.333333  0.0  0.0  0.0  0.0  0.0
 0.0   0.0       0.0   0.0       0.0  0.0  0.0  0.0  0.0
 0.0   0.0       0.0   0.0       0.0  0.0  0.0  0.0  0.0
 0.0   0.0       0.0   0.0       0.0  0.0  0.0  0.0  0.0
 0.0   0.0       0.0   0.0       0.0  0.0  0.0  0.0  0.0
9-element Vector{Float64}:
 0.1
 0.0
 0.1
 0.4
 0.4
 0.0
 0.0
 0.0
 0.0

In [37]:
rockNoGhostBMatrix = [0.1e0 1//3 1//3 1; 0 1//3 0 0; 0.45e0 1//3 1//3 0; 0.45e0 0 1//3 0]

rockNoGhostBInitial = [0.1e0,0, 0,0.5e0,0.4e0, 0, 0, 0, 0]

rockNoGhostB = AugMatrix([rockNoGhostBMatrix[1:2, 1:2] zeros(2,1) rockNoGhostBMatrix[1:2, 3:4] zeros(2, 4)
    zeros(1, 9)
    rockNoGhostBMatrix[3:4, 1:2] zeros(2,1) rockNoGhostBMatrix[3:4, 3:4] zeros(2, 4)
    zeros(4, 9)
    ],
    rockNoGhostBInitial)

9×9 Matrix{Float64}:
 0.1   0.333333  0.0  0.333333  1.0  0.0  0.0  0.0  0.0
 0.0   0.333333  0.0  0.0       0.0  0.0  0.0  0.0  0.0
 0.0   0.0       0.0  0.0       0.0  0.0  0.0  0.0  0.0
 0.45  0.333333  0.0  0.333333  0.0  0.0  0.0  0.0  0.0
 0.45  0.0       0.0  0.333333  0.0  0.0  0.0  0.0  0.0
 0.0   0.0       0.0  0.0       0.0  0.0  0.0  0.0  0.0
 0.0   0.0       0.0  0.0       0.0  0.0  0.0  0.0  0.0
 0.0   0.0       0.0  0.0       0.0  0.0  0.0  0.0  0.0
 0.0   0.0       0.0  0.0       0.0  0.0  0.0  0.0  0.0
9-element Vector{Float64}:
 0.1
 0.0
 0.0
 0.5
 0.4
 0.0
 0.0
 0.0
 0.0

In [124]:
function lookup16ride(; variation="none")::Matrix
    r = Matrix(undef, 4, 4)
    r[1,1] = ["!N![_e","[","!N![_e","[","!N![_ec","[","!N![_e","[","!N![_e","[","!N![_e","[","!N![_ec","[","!N![_e","["] #hihat eighths, trash
    #NB Windows midi doesn't have a good way of making a trash hat?  !N! defined as tenuto mark and lowered volume in makeppheader
    r[2,1] = ["[^e", "[^e", "[^e", "[^e", "[^ec", "[^e", "[^e", "[^e", "[^e", "[^e", "[^e", "[^e", "[^ec", "[^e", "[^e", "[^e"]; #hihat 16
    r[3,1] = ["!>![^e", "!U![^e", "!>![^e", "!U![^e", "!>![^ec", "!U![^e","!>![^e", "!U![^e","!>![^e", "!U![^e", "!>![^e", "!U![^e", "!>![^ec", "!U![^e", "!>![^e", "!U![^e"]; #drive 16
    if variation == "funk"
        r[4,1] = ["[^e", "[^e", "[^e", "[_e", "[^ec", "[^e", "[^e", "[^e", "[^e", "[^e", "[^e", "[_e", "[^ec", "[^e", "[^e", "[^e"]  #"James Brown" slides #7
    elseif variation == "disco"
         r[4,1] = ["[^e", "[^e", "[_e", "[^e", "[^ec", "[^e", "[_e", "[^e", "[^e", "[^e", "[_e", "[^e", "[^ec", "[^e", "[_e", "[^e"] #slide every and #6
    else
        r[4,1] = ["[^e", "[^e", "[^e", "[^e", "[^ec", "[^e", "[^e", "[^e", "[^e", "[^e", "[^e", "[^e", "[^ec", "[^e", "[^e", "[_e"];  #one slide 16
    end
    r[1,2] = ["[^g","[","[^g","[","[^gc","[","[^g","[","[^g","[","[^g","[","[^gc","[","[^g","["] #ride quarter
    r[2,2] = ["[^g", "[^g", "[^g", "[^g", "[^gc", "[^g", "[^g", "[^g", "[^g", "[^g", "[^g", "[^g", "[^gc", "[^g", "[^g", "[^g"]; #ride 16 
    r[3,2] = ["[^g^D", "[^g", "[^g", "[^g", "[^g^Dc", "[^g", "[^g", "[^g", "[^g^D", "[^g", "[^g", "[^g", "[^g^Dc", "[^g", "[^g", "[^g"]; #ride 16 with step beats
    r[4,2] = ["[^g", "[^g", "[^g^D", "[^g", "[^gc", "[^g", "[^g^D", "[^g", "[^g", "[^g", "[^g^D", "[^g", "[^gc", "[^g", "[^g^D", "[^g" ]; #ride 16 with step ands
    r[1,3] = ["[g","[","[g","[","[gc","[","[g","[","[g","[","[g","[","[gc","[","[g","["] #bell 8
    r[2,3] = ["[g","[g","[","[","[gc","[g","[","[","[g","[g","[","[","[gc","[g","[","["] #bell 16th pairs on beat
    r[3,3] = ["[g","!U![^g","[g","!U![^g","[gc","!U![^g","[g","!U![^g","[g","!U![^g","[g","!U![^g","[gc","!U![^g","[g","!U![^g"] #bell drive, offbeat ride
    r[4,3] = ["[","[","[g","[g","[c","[","[g","[g","[","[","[g","[g","[c","[","[g","[g"] #bell pairs on and
    r[1,4] = ["[^D","[","[","[","[^Dc","[","[","[","[^D","[","[","[","[^Dc","[","[","["]  #step Q
    r[2,4] = ["[^D","[","[^D","[","[^Dc","[","[^D","[","[^D","[","[^D","[","[^Dc","[","[^D","["] #step 8
    r[3,4] = ["[g^D","[g","[^D","[","[g^Dc","[g","[^D","[","[g^D","[g","[^D","[","[g^Dc","[g","[^D","["] #step 8 with bell beat pair
    r[4,4] = ["[^D","[","[g^D","[g","[^Dc","[","[g^D","[g","[^D","[","[g^D","[g","[^Dc","[","[g^D","[g"] #step 8 with bell and pair
    return(r)
end


lookup16ride (generic function with 1 method)

In [118]:
#input a sixteenth note bass rhythm and an 16th note ride pattern, output abc notation
function abctime16patinst16ghost(L, pattern, instr; variation="none")
    n = length(L)
    r = lookup16ride(variation=variation)
    c = ["]/2", "F]/2", "!Q!c]/2"] #nothing, kick, ghost snare
    m = [c[L[i] + 1] for i = 1:n]
    s = ""
    for i = 1:n
        s = s * r[pattern, instr][(i - 1) % 16 + 1] * m[i]
        if i % 4 == 0
            s = s * " "
        end
        if i % 16 == 0
            s = s * "|\n"
        end
    end
    return(s)
end


abctime16patinst16ghost (generic function with 1 method)

In [40]:
#output linear fill in $beats number of sixteenth notes
function fill16gen(
  beats::Int,
  trans::AugDblMatrix)
    nextv = trans.init
    L = []
    for j = 1:beats
        current = markovrand(nextv)
        append!(L,current)
        if j % 2 == 1
            nextv = trans.mat1[1:6, current + 1]
        else
            nextv = trans.mat2[1:6, current + 1]
        end
    end
    return(L)
end


fill16gen (generic function with 1 method)

In [41]:
rockFillLeftToRight = [
    0.2e0 0.1e0 0.2e0 0 0.3e0 0.2e0; 
    0.4e0 0.2e0 0 0.3e0 0.3e0 0.2e0; 
    0.1e0 0.4e0 0.2e0 0.1e0 0.1e0 0.2e0; 
    0.1e0 0.1e0 0.4e0 0.3e0 0.1e0 0.2e0; 
    0.1e0 0.1e0 0.2e0 0.2e0 0.1e0 0.2e0; 
    0.1e0 0.1e0 0 0.1e0 0.1e0 0]

rockFillRightToLeft = [
    0.4e0 0.2e0 0.1e0 0.1e0 0.3e0 0
    0.2e0 0.4e0 0.2e0 0.2e0 0.2e0 0
    0.1e0 0 0.4e0 0.1e0 0.1e0 0
    0 0.1e0 0.1e0 0.4e0 0.1e0 0
    0.2e0 0.1e0 0.1e0 0.1e0 0.2e0 0
    0.1e0 0.2e0 0.1e0 0.1e0 0.1e0 0.10e1]


rockFillInitial = [0.4e0,0.4e0,0.1e0,0.1e0,0,0] #snare, high, mid, low, kick, rest

rockFill= AugDblMatrix(rockFillLeftToRight, rockFillRightToLeft, rockFillInitial)

6×6 Matrix{Float64}:
 0.2  0.1  0.2  0.0  0.3  0.2
 0.4  0.2  0.0  0.3  0.3  0.2
 0.1  0.4  0.2  0.1  0.1  0.2
 0.1  0.1  0.4  0.3  0.1  0.2
 0.1  0.1  0.2  0.2  0.1  0.2
 0.1  0.1  0.0  0.1  0.1  0.0
6×6 Matrix{Float64}:
 0.4  0.2  0.1  0.1  0.3  0.0
 0.2  0.4  0.2  0.2  0.2  0.0
 0.1  0.0  0.4  0.1  0.1  0.0
 0.0  0.1  0.1  0.4  0.1  0.0
 0.2  0.1  0.1  0.1  0.2  0.0
 0.1  0.2  0.1  0.1  0.1  1.0
6-element Vector{Float64}:
 0.4
 0.4
 0.1
 0.1
 0.0
 0.0

In [42]:
#input a sixteenth note fill and a crescendo/descr indicator, output abc notation
function abcfill16(L, crdc)
    n = length(L)
    c = ["c/2","e/2","A/2","G/2","F/2","z/2"]
    m = [c[L[i] + 1] for i = 1:n]
    s = ""
    for i = 1:n
        s = s * m[i]
        if i % 4 == 0
            s = s * " "
        end
        if i % 16 == 12
            if crdc < 0
                s = s * "!>(!"
            elseif 0 < crdc
                s = s * "!<(!"
            end
        end
        if i % 16 == 14
            if crdc < 0
                s = s * "!>)!"
            elseif 0 < crdc
                s = s * "!<)!"
            end
        end
    end
    return(s)
end


abcfill16 (generic function with 1 method)

In [43]:
dynMatrix = [
    0 0 0.2e0 0.2e0 0.1e0 0.1e0 0 0 0 0 0 0; 
    0 0 0.2e0 0.1e0 0.1e0 0.1e0 0 0 0 0 0 0; 
    0.2e0 0.2e0 0 0 0.2e0 0.1e0 0.1e0 0.1e0 0.1e0 0.2e0 0.1e0 0.2e0; 
    0.2e0 0.2e0 0 0 0.2e0 0.1e0 0.1e0 0.1e0 0.2e0 0.1e0 0.2e0 0.1e0; 
    0 0 0.1e0 0.2e0 0 0 0.2e0 0.1e0 0 0 0.1e0 0.2e0; 
    0 0 0.2e0 0.2e0 0 0 0.2e0 0.1e0 0 0 0.2e0 0.1e0; 
    0.1e0 0.2e0 0 0 0.1e0 0.2e0 0 0 0.2e0 0.2e0 0 0; 
    0.2e0 0.1e0 0 0 0.1e0 0.2e0 0 0 0.2e0 0.1e0 0 0; 
    0.1e0 0.2e0 0.1e0 0.2e0 0.1e0 0.1e0 0.1e0 0.2e0 0 0 0.2e0 0.2e0;
    0.2e0 0.1e0 0.2e0 0.1e0 0.1e0 0.1e0 0.1e0 0.2e0 0 0 0.2e0 0.2e0; 
    0 0 0 0 0 0 0.1e0 0.1e0 0.1e0 0.2e0 0 0; 
    0 0 0 0 0 0 0.1e0 0.1e0 0.2e0 0.2e0 0 0]


dynFinal = [
    0.1e0 0.2e0 0.2e0 0.2e0 0.1e0 0.1e0 0.1e0 0.1e0 0.1e0 0.2e0 0 0; 
    0.2e0 0.1e0 0.2e0 0.1e0 0.1e0 0.1e0 0.1e0 0.1e0 0.2e0 0.1e0 0 0; 
    0.2e0 0.2e0 0 0 0.2e0 0.1e0 0.2e0 0.1e0 0.2e0 0.2e0 0.1e0 0.2e0; 
    0.2e0 0.2e0 0 0 0.2e0 0.1e0 0.2e0 0.1e0 0.2e0 0.1e0 0.2e0 0.1e0; 
    0 0 0 0 0 0 0 0 0 0 0 0; 
    0 0 0 0 0 0 0 0 0 0 0 0; 
    0 0 0 0 0 0 0 0 0 0 0 0; 
    0 0 0 0 0 0 0 0 0 0 0 0; 
    0.1e0 0.2e0 0.1e0 0.2e0 0.1e0 0.2e0 0.1e0 0.2e0 0 0 0.2e0 0.2e0; 
    0.2e0 0.1e0 0.2e0 0.2e0 0.1e0 0.2e0 0.1e0 0.2e0 0 0 0.2e0 0.2e0; 
    0 0 0.1e0 0.2e0 0.1e0 0.1e0 0.1e0 0.1e0 0.1e0 0.2e0 0.1e0 0.2e0; 
    0 0 0.2e0 0.1e0 0.1e0 0.1e0 0.1e0 0.1e0 0.2e0 0.2e0 0.2e0 0.1e0]


dynInitial = [0.5e-1,0.5e-1,0.1e0,0.1e0,0.1e0,0.1e0,0.1e0,0.1e0,0.1e0,0.1e0,0.5e-1,0.5e-1]
#pp>, pp<, p>, p<, mp>, mp<, mf>, mf<, f>, f<, ff>, ff<

dyn = AugDblMatrix(dynMatrix, dynFinal, dynInitial)

12×12 Matrix{Float64}:
 0.0  0.0  0.2  0.2  0.1  0.1  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.2  0.1  0.1  0.1  0.0  0.0  0.0  0.0  0.0  0.0
 0.2  0.2  0.0  0.0  0.2  0.1  0.1  0.1  0.1  0.2  0.1  0.2
 0.2  0.2  0.0  0.0  0.2  0.1  0.1  0.1  0.2  0.1  0.2  0.1
 0.0  0.0  0.1  0.2  0.0  0.0  0.2  0.1  0.0  0.0  0.1  0.2
 0.0  0.0  0.2  0.2  0.0  0.0  0.2  0.1  0.0  0.0  0.2  0.1
 0.1  0.2  0.0  0.0  0.1  0.2  0.0  0.0  0.2  0.2  0.0  0.0
 0.2  0.1  0.0  0.0  0.1  0.2  0.0  0.0  0.2  0.1  0.0  0.0
 0.1  0.2  0.1  0.2  0.1  0.1  0.1  0.2  0.0  0.0  0.2  0.2
 0.2  0.1  0.2  0.1  0.1  0.1  0.1  0.2  0.0  0.0  0.2  0.2
 0.0  0.0  0.0  0.0  0.0  0.0  0.1  0.1  0.1  0.2  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.1  0.1  0.2  0.2  0.0  0.0
12×12 Matrix{Float64}:
 0.1  0.2  0.2  0.2  0.1  0.1  0.1  0.1  0.1  0.2  0.0  0.0
 0.2  0.1  0.2  0.1  0.1  0.1  0.1  0.1  0.2  0.1  0.0  0.0
 0.2  0.2  0.0  0.0  0.2  0.1  0.2  0.1  0.2  0.2  0.1  0.2
 0.2  0.2  0.0  0.0  0.2  0.1  0.2  0.1  0.2  0.1  0.2

In [44]:
rockPatMatrix = [
    #8  Q   b   +
    0   0.4 0.4 0.4 #8 
    0.4 0   0.2 0.2   #Q
    0.2 0.2 0   0.4 #b
    0.4 0.4 0.4 0 #+
    ]



rockPatInitial = [0.4; 0.4; 0.2; 0]

rockPat = AugMatrix(rockPatMatrix, rockPatInitial)

4×4 Matrix{Float64}:
 0.0  0.4  0.4  0.4
 0.4  0.0  0.2  0.2
 0.2  0.2  0.0  0.4
 0.4  0.4  0.4  0.0
4-element Vector{Float64}:
 0.4
 0.4
 0.2
 0.0

In [45]:
rockInstMatrix = [
   #HH  RC  RB  ST 
    0.1 0.4 0.4   0.4   #HH
    0.6 0.1 0.4   0.4   #RC
    0.2 0.4 0.1   0.2   #RB
    0.1 0.1 0.1   0     #ST
]

rockInstInitial = [0.5; 0.3; 0; 0.2]

rockInst = AugMatrix(rockInstMatrix, rockInstInitial)

4×4 Matrix{Float64}:
 0.1  0.4  0.4  0.4
 0.6  0.1  0.4  0.4
 0.2  0.4  0.1  0.2
 0.1  0.1  0.1  0.0
4-element Vector{Float64}:
 0.5
 0.3
 0.0
 0.2

In [110]:
#output time, fill, time, fill with cresc/decr in abc notation
function abctune16patinst16ghost(
  vol,
  bass,
  pat,
  inst,
  fill,
  crdc; 
        variation="none")
    fp = ["!pp!","!p!","!mp!","!mf!","!f!","!ff!"]
    return("|:" * fp[vol] * abctime16patinst16ghost(bass, pat, inst, variation=variation) * "[1" * abcfill16(fill, 0) * ":|2" * abcfill16(fill, crdc)* "\n")
end


abctune16patinst16ghost (generic function with 1 method)

In [111]:
#put it all together, output abc notation
import Dates
function abcsolo16patinst16ghost(
  len::Int,
  basstrans::AugMatrix,
  filltrans::AugDblMatrix,
  dyntrans::AugDblMatrix,
  pattrans::AugMatrix,
  insttrans::AugMatrix,
  feel::String;
     variation="rock"  )
    s = ""
    curdyn = markovrand(dyn.init)
    curpat = markovrand(pattrans.init)
    curinst = markovrand(insttrans.init)
    for i = 1:len
        vol = div(curdyn, 2) + 1
        bass = ghostbassgen(24, basstrans)
        pat = curpat + 1
        inst = curinst + 1
        fill = fill16gen(16, filltrans)
        crdc = 2 * mod(curdyn, 2) - 1
        s = s * abctune16patinst16ghost(vol, bass, pat, inst, fill, crdc, variation=variation)
        if i + 1 < len
            curdyn = markovrand(dyntrans.mat1[:, curdyn + 1])
        else
            curdyn = markovrand(dyntrans.mat2[:, curdyn + 1])
        end
        curpat = markovrand(pattrans.mat[:, curpat + 1])
        curinst = markovrand(insttrans.mat[:, curinst + 1])
    end
    s = insert(s, findlast(occursin("acbdefgABCDEFG"), s)-1, "!fermata!") #final fermata
    s = replace(s, "[]" => "z") #rest
    s = makeppheader("Computer-generated \\`Etude for Solo Drum Kit", String(@functionName) * "/" * variation, "$(Dates.now())", "4/4", feel, 80) * s * "|]\n"
    return(s)
end


abcsolo16patinst16ghost (generic function with 1 method)

In [112]:
solo = abcsolo16patinst16ghost(4, rockBass, rockFill, dyn, rockPat, rockInst, "Rock 16th feel")
if true
    print(solo)
end

%%abc-include percussions-JBH.abh

I:linebreak $

#ifdef MIDI
#define !>! "[I:volinc 50]"
#define !Q! "[I:vol 20]" 
                %%%ghost
#define !U! "[I:vol 40]" 
                %%%unaccent
#define !N! "[I:volinc -25]" 
            %%%tenuto for trash hat
#else
#define !N! "\"^_\"" 
                %%%tenuto
#define !Q! !(.!!).! 
                %%%ghost
#define !U! !anti!  
                %%%unaccent
#endif

X:1
T:Computer-generated \`Etude for Solo Drum Kit
C:#abcsolo16patinst16ghost#29/rock
O:2021-12-31T17:45:42.529
M:4/4
L:1/8
Q:"Rock 16th feel" 1/4=80
K:none clef=perc
[V:1 clef=perc, stem=up]     % activate abc2xml.py map
%%voicemap drummap  % activate abcm2ps/abc2svg map
%%MIDI channel 10   % activate abc2midi map
%%MIDI program 0
%%flatbeams
%%propagate-accidentals not
%%pos ornament up
%%ornament up
%%MIDI fermatafixed
|:!mf![g^D]/2[g!Q!c]/2[^DF]/2z/2 [g^Dc]/2[g]/2[^DF]/2[F]/2 [g^D]/2[g!Q!c]/2[^DF]/2[F]/2 [g^Dc]/2[g]/2[^DF]/2z/2 |
[g^D]/2[g!Q!c]/2[^DF]/2z/2 [g^DcF]/2[g]/2

In [79]:
#write a string to a file
open(f->print(f, solo), "rockghosttest.abp", "w")

In [82]:
#Converts ABP file to score and MIDI and displays/plays it
abptest("rockghosttest.abp")

abcm2ps-8.14.12 (2021-07-14)
File rockghosttest-ps.abc
File percussions-JBH.abh
Output written on rockghosttest.xhtml (1 page, 1 title, 61145 bytes)
4.59 June 27 2021 abc2midi
writing MIDI file rockghosttest.mid
Warning in line-char 41-0 : Track 1 Bar 19 has 9/2 units instead of 4 in repeat


rockghosttest-ps.abc:28:96: warning: Line overfull (637pt of 628pt)


Process(`cmd /C start rockghosttest.xhtml`, ProcessExited(0))

In [81]:
NoGhostSolo = abcsolo16patinst16ghost(4, rockNoGhostB, rockFill, dyn, rockPat, rockInst, "Rock 16th feel")
print(NoGhostSolo)

%%abc-include percussions-JBH.abh

I:linebreak $

#ifdef MIDI
#define !>! "[I:volinc 50]"
#define !Q! "[I:vol 20]" 
                %%%ghost
#define !U! "[I:vol 40]" 
                %%%unaccent
#define !N! "[I:volinc -25]" 
            %%%tenuto for trash hat
#else
#define !N! "\"^_\"" 
                %%%tenuto
#define !Q! !(.!!).! 
                %%%ghost
#define !U! !anti!  
                %%%unaccent
#endif

X:1
T:Computer-generated \`Etude for Solo Drum Kit
C:abcsolo16patinst16ghost
O:2021-12-31T17:24:54.329
M:4/4
L:1/8
Q:"Rock 16th feel" 1/4=80
K:none clef=perc
[V:1 clef=perc, stem=up]     % activate abc2xml.py map
%%voicemap drummap  % activate abcm2ps/abc2svg map
%%MIDI channel 10   % activate abc2midi map
%%MIDI program 0
%%flatbeams
%%propagate-accidentals not
%%pos ornament up
%%ornament up
%%MIDI fermatafixed
|:!mp![^eF]/2[^e]/2[^eF]/2[^e]/2 [^ecF]/2[^e]/2[^eF]/2[^e]/2 [^e]/2[^e]/2[^eF]/2[^eF]/2 [^ec]/2[^e]/2[^eF]/2[^e]/2 |
[^eF]/2[^eF]/2[^e]/2[^e]/2 [^ecF]/2[^e]/2[^e]/2

In [109]:
#write a string to a file
open(f->print(f, NoGhostSolo), "rocknoghosttest.abp", "w")
#Converts ABP file to score and MIDI and displays/plays it
abptest("rocknoghosttest.abp")

abcm2ps-8.14.12 (2021-07-14)
File rocknoghosttest-ps.abc
File percussions-JBH.abh
Output written on rocknoghosttest.xhtml (1 page, 1 title, 54401 bytes)
4.59 June 27 2021 abc2midi
Warning in line-char 24-5 : instruction !N! ignored
Warning in line-char 24-20 : instruction !N! ignored
Warning in line-char 24-33 : instruction !N! ignored
Warning in line-char 24-47 : instruction !N! ignored
Warning in line-char 24-60 : instruction !N! ignored
Warning in line-char 24-73 : instruction !N! ignored
Warning in line-char 24-89 : instruction !N! ignored
Warning in line-char 24-102 : instruction !N! ignored
Warning in line-char 25-0 : instruction !N! ignored
Warning in line-char 25-12 : instruction !N! ignored
Warning in line-char 25-25 : instruction !N! ignored
Warning in line-char 25-41 : instruction !N! ignored
Warning in line-char 25-54 : instruction !N! ignored
Warning in line-char 25-67 : instruction !N! ignored
Warning in line-char 25-80 : instruction !N! ignored
Warning in line-char 25-94

rocknoghosttest-ps.abc:29:47: warning: Line overfull (635pt of 628pt)


Process(`cmd /C start rocknoghosttest.xhtml`, ProcessExited(0))

In [24]:
#0 = rest, 1 = bass, 2 = LH
                #00    #01  #02   #10  #11   #12  #zeros
funkBassMatrix = [0.1e0 0.2  0     0.1  1//3 0   0 0 0 #00
                    0.2 0.2  0.2   0.2  1//3 0.1    0 0 0  #01
                    0.2 0.2  0.2   0.2  1//3 0.1    0 0 0  #02
                    0.1 0.2  0.2   0.1  0    0.3 0 0 0  #10
                    0.2 0    0.2   0.2  0    0.2 0 0 0  #11
                    0.2 0.2  0.2   0.2  0    0.3 0 0 0    #12
                    zeros(3,9)]                 #zeros

funkBassInitial = [0.1, 0.2, 0.1, 0.2, 0.2, 0.2, 0, 0, 0]


funkBass = AugMatrix(funkBassMatrix, funkBassInitial)

9×9 Matrix{Float64}:
 0.1  0.2  0.0  0.1  0.333333  0.0  0.0  0.0  0.0
 0.2  0.2  0.2  0.2  0.333333  0.1  0.0  0.0  0.0
 0.2  0.2  0.2  0.2  0.333333  0.1  0.0  0.0  0.0
 0.1  0.2  0.2  0.1  0.0       0.3  0.0  0.0  0.0
 0.2  0.0  0.2  0.2  0.0       0.2  0.0  0.0  0.0
 0.2  0.2  0.2  0.2  0.0       0.3  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0       0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0       0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0       0.0  0.0  0.0  0.0
9-element Vector{Float64}:
 0.1
 0.2
 0.1
 0.2
 0.2
 0.2
 0.0
 0.0
 0.0

In [25]:
funkNoGhostBMatrix = [0.1e0 1//3 0.25e0 0.5e0; 0.3e0 1//3 0.25e0 0.5e0; 0.3e0 1//3 0.25e0 0; 0.3e0 0 0.25e0 0]

funkNoGhostBInitial = [0.1e0, 0.1e0, 0, 0.4e0, 0.4e0, 0, 0, 0, 0 ]

funkNoGhostB = AugMatrix([funkNoGhostBMatrix[1:2, 1:2] zeros(2,1) funkNoGhostBMatrix[1:2, 3:4] zeros(2, 4)
    zeros(1, 9)
    funkNoGhostBMatrix[3:4, 1:2] zeros(2,1) funkNoGhostBMatrix[3:4, 3:4] zeros(2, 4)
    zeros(4, 9)
    ],
    funkNoGhostBInitial)

9×9 Matrix{Float64}:
 0.1  0.333333  0.0  0.25  0.5  0.0  0.0  0.0  0.0
 0.3  0.333333  0.0  0.25  0.5  0.0  0.0  0.0  0.0
 0.0  0.0       0.0  0.0   0.0  0.0  0.0  0.0  0.0
 0.3  0.333333  0.0  0.25  0.0  0.0  0.0  0.0  0.0
 0.3  0.0       0.0  0.25  0.0  0.0  0.0  0.0  0.0
 0.0  0.0       0.0  0.0   0.0  0.0  0.0  0.0  0.0
 0.0  0.0       0.0  0.0   0.0  0.0  0.0  0.0  0.0
 0.0  0.0       0.0  0.0   0.0  0.0  0.0  0.0  0.0
 0.0  0.0       0.0  0.0   0.0  0.0  0.0  0.0  0.0
9-element Vector{Float64}:
 0.1
 0.1
 0.0
 0.4
 0.4
 0.0
 0.0
 0.0
 0.0

In [26]:
funkFillLeftToRight = [0.2e0 0.1e0 0.1e0 0 0.3e0 0.2e0; 0.4e0 0.2e0 0 0.3e0 0.3e0 0.2e0; 0.1e0 0.4e0 0.2e0 0.1e0 0.1e0 0.2e0; 0.1e0 0.1e0 0.4e0 0.3e0 0.1e0 0.2e0; 0.1e0 0.1e0 0.1e0 0.2e0 0.1e0 0.1e0; 0.1e0 0.1e0 0.2e0 0.1e0 0.1e0 0.1e0]

funkFillRightToLeft = [0.4e0 0.2e0 0.1e0 0.1e0 0.3e0 0.2e0; 0.2e0 0.4e0 0.2e0 0.2e0 0.2e0 0.2e0; 0.1e0 0 0.4e0 0.1e0 0.1e0 0.2e0; 0 0.1e0 0.1e0 0.4e0 0.1e0 0.2e0; 0.2e0 0.1e0 0.1e0 0.1e0 0.2e0 0.1e0; 0.1e0 0.2e0 0.1e0 0.1e0 0.1e0 0.1e0]


funkFillInitial = [0.3e0,0.3e0,0.1e0,0.1e0,0.1e0,0.1e0]

funkFill = AugDblMatrix(funkFillLeftToRight, funkFillRightToLeft, funkFillInitial)

6×6 Matrix{Float64}:
 0.2  0.1  0.1  0.0  0.3  0.2
 0.4  0.2  0.0  0.3  0.3  0.2
 0.1  0.4  0.2  0.1  0.1  0.2
 0.1  0.1  0.4  0.3  0.1  0.2
 0.1  0.1  0.1  0.2  0.1  0.1
 0.1  0.1  0.2  0.1  0.1  0.1
6×6 Matrix{Float64}:
 0.4  0.2  0.1  0.1  0.3  0.2
 0.2  0.4  0.2  0.2  0.2  0.2
 0.1  0.0  0.4  0.1  0.1  0.2
 0.0  0.1  0.1  0.4  0.1  0.2
 0.2  0.1  0.1  0.1  0.2  0.1
 0.1  0.2  0.1  0.1  0.1  0.1
6-element Vector{Float64}:
 0.3
 0.3
 0.1
 0.1
 0.1
 0.1

In [27]:
funkPatMatrix = [
    #8  Q   b   +
    0   0.2 0.2 0.2 #8 
    0.2 0   0.4 0.4   #Q
    0.2 0.2 0   0.4 #b
    0.6 0.6 0.4 0 #+
    ]

funkPatInitial = [0; 0.2; 0.4; 0.4]

funkPat = AugMatrix(funkPatMatrix, funkPatInitial)

4×4 Matrix{Float64}:
 0.0  0.2  0.2  0.2
 0.2  0.0  0.4  0.4
 0.2  0.2  0.0  0.4
 0.6  0.6  0.4  0.0
4-element Vector{Float64}:
 0.0
 0.2
 0.4
 0.4

In [28]:
funkInstMatrix = [
   #HH  RC  RB  ST 
    0.1 0.4 0.4   0.4   #HH
    0.6 0.1 0.4   0.4   #RC
    0.2 0.4 0.1   0.2   #RB
    0.1 0.1 0.1   0     #ST
]

funkInstInitial = [0.4; 0.4; 0.2; 0]

funkInst = AugMatrix(funkInstMatrix, funkInstInitial)

4×4 Matrix{Float64}:
 0.1  0.4  0.4  0.4
 0.6  0.1  0.4  0.4
 0.2  0.4  0.1  0.2
 0.1  0.1  0.1  0.0
4-element Vector{Float64}:
 0.4
 0.4
 0.2
 0.0

In [129]:
funksolo = abcsolo16patinst16ghost(4, funkBass, funkFill, dyn, funkPat, funkInst, "Funk 16th feel", variation="funk")
print(funksolo)

%%abc-include percussions-JBH.abh

I:linebreak $

#ifdef MIDI
#define !>! "[I:volinc 50]"
#define !Q! "[I:vol 20]" 
                %%%ghost
#define !U! "[I:vol 40]" 
                %%%unaccent
#define !N! "[I:volinc -25]" 
            %%%tenuto for trash hat
#else
#define !N! "\"^_\"" 
                %%%tenuto
#define !Q! !(.!!).! 
                %%%ghost
#define !U! !anti!  
                %%%unaccent
#endif

X:1
T:Computer-generated \`Etude for Solo Drum Kit
C:#abcsolo16patinst16ghost#29/funk
O:2021-12-31T17:56:33.722
M:4/4
L:1/8
Q:"Funk 16th feel" 1/4=80
K:none clef=perc
[V:1 clef=perc, stem=up]     % activate abc2xml.py map
%%voicemap drummap  % activate abcm2ps/abc2svg map
%%MIDI channel 10   % activate abc2midi map
%%MIDI program 0
%%flatbeams
%%propagate-accidentals not
%%pos ornament up
%%ornament up
%%MIDI fermatafixed
|:!mp![^e]/2[^eF]/2[^e]/2[_e!Q!c]/2 [^ec]/2[^eF]/2[^e]/2[^eF]/2 [^e]/2[^eF]/2[^e]/2[_eF]/2 [^ecF]/2[^e!Q!c]/2[^e]/2[^eF]/2 |
[^eF]/2[^e!Q!c]/2[^eF]/2[_eF]/

In [130]:
#write a string to a file
open(f->print(f, funksolo), "funkghosttest.abp", "w")

#Converts ABP file to score and MIDI and displays/plays it
abptest("funkghosttest.abp")

abcm2ps-8.14.12 (2021-07-14)
File funkghosttest-ps.abc
File percussions-JBH.abh
Output written on funkghosttest.xhtml (1 page, 1 title, 63563 bytes)


funkghosttest-ps.abc:29:28: warning: Line overfull (635pt of 628pt)


4.59 June 27 2021 abc2midi
writing MIDI file funkghosttest.mid
Warning in line-char 41-0 : Track 1 Bar 19 has 9/2 units instead of 4 in repeat


Process(`cmd /C start funkghosttest.xhtml`, ProcessExited(0))

In [113]:
funkNoGhostsolo = abcsolo16patinst16ghost(4, funkNoGhostB, funkFill, dyn, funkPat, funkInst, "Funk 16th feel")
print(funkNoGhostsolo)

%%abc-include percussions-JBH.abh

I:linebreak $

#ifdef MIDI
#define !>! "[I:volinc 50]"
#define !Q! "[I:vol 20]" 
                %%%ghost
#define !U! "[I:volinc -20]" 
                %%%unaccent
#else
#define !N! "\"^_\"" 
                %%%tenuto
#define !Q! !(.!!).! 
                %%%ghost
#define !U! !anti!  
                %%%unaccent
#endif

X:1
T:Computer-generated \`Etude for Solo Drum Kit
C:abcsolo16patinst16ghost
O:2021-11-22T19:04:01.226
M:4/4
L:1/8
Q:"Funk 16th feel" 1/4=80
K:none clef=perc
[V:1 clef=perc, stem=up]     % activate abc2xml.py map
%%voicemap drummap  % activate abcm2ps/abc2svg map
%%MIDI channel 10   % activate abc2midi map
%%MIDI program 0
%%flatbeams
%%propagate-accidentals not
%%pos ornament up
%%ornament up
%%MIDI fermatafixed
|:!f!!>![^eF]/2!U![^e]/2!>![^eF]/2!U![^e]/2 !>![c^e]/2!U![^e]/2!>![^eF]/2!U![^e]/2 !>![^e]/2!U![^eF]/2!>![^eF]/2!U![^e]/2 !>![c^eF]/2!U![^e]/2!>![^eF]/2!U![^e]/2 |
!>![^e]/2!U![^e]/2!>![^eF]/2!U![^e]/2 !>![c^e]/2!U![^eF]/2!>![

In [117]:
#write a string to a file
open(f->print(f, funkNoGhostsolo), "funknoghosttest.abp", "w")

In [83]:
#Converts ABP file to score and MIDI and displays/plays it
abptest("funknoghosttest.abp")

abcm2ps-8.14.12 (2021-07-14)
File funknoghosttest-ps.abc
File percussions-JBH.abh
Output written on funknoghosttest.xhtml (1 page, 1 title, 58857 bytes)


funknoghosttest-ps.abc:29:47: warning: Line overfull (635pt of 628pt)


4.59 June 27 2021 abc2midi
writing MIDI file funknoghosttest.mid
Warning in line-char 40-0 : Track 1 Bar 19 has 9/2 units instead of 4 in repeat


Process(`cmd /C start funknoghosttest.xhtml`, ProcessExited(0))

In [84]:
#0 = rest, 1 = bass
discoNoGhostBMatrix = 
    #00   #01  #10      #11
    [0    0    0.5   0.5; 
    0     0    0.5   0.5; 
    0.5   1    0      0; 
    0.5   0    0      0]

discoNoGhostBInitial = [0, 0, 0, 0.5, 0.5, 0, 0, 0, 0 ]

discoNoGhostB = AugMatrix([discoNoGhostBMatrix[1:2, 1:2] zeros(2,1) discoNoGhostBMatrix[1:2, 3:4] zeros(2, 4)
    zeros(1, 9)
    discoNoGhostBMatrix[3:4, 1:2] zeros(2,1) discoNoGhostBMatrix[3:4, 3:4] zeros(2, 4)
    zeros(4, 9)
    ],
    discoNoGhostBInitial)

9×9 Matrix{Float64}:
 0.0  0.0  0.0  0.5  0.5  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.5  0.5  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.5  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.5  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
9-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.5
 0.5
 0.0
 0.0
 0.0
 0.0

In [85]:
discoPatMatrix = [
    #Q  8   b   +
    0   0.2 0.2 0.2 #Q 
    0.2 0   0.4 0.4   #8
    0.2 0.2 0   0.4 #b
    0.6 0.6 0.4 0 #+
    ]

discoPatInitial = [0; 0.5; 0.5; 0]

discoPat = AugMatrix(discoPatMatrix, discoPatInitial)

4×4 Matrix{Float64}:
 0.0  0.2  0.2  0.2
 0.2  0.0  0.4  0.4
 0.2  0.2  0.0  0.4
 0.6  0.6  0.4  0.0
4-element Vector{Float64}:
 0.0
 0.5
 0.5
 0.0

In [86]:
discoInstMatrix = [
   #HH  RC  RB  ST 
    0.1 0.5 0.6   0.4   #HH
    0.3 0.1 0.3   0.2   #RC
    0.6 0.4 0.1   0.4   #RB
    0    0  0   0     #ST
]

discoInstInitial = [0.6; 0.2; 0.2; 0]

discoInst = AugMatrix(discoInstMatrix, discoInstInitial)

4×4 Matrix{Float64}:
 0.1  0.5  0.6  0.4
 0.3  0.1  0.3  0.2
 0.6  0.4  0.1  0.4
 0.0  0.0  0.0  0.0
4-element Vector{Float64}:
 0.6
 0.2
 0.2
 0.0

In [87]:
discoFillLeftToRight = [0.2e0 0.1e0 0.1e0 0 0.3e0 0.2e0; 0.4e0 0.2e0 0 0.3e0 0.3e0 0.2e0; 0.1e0 0.4e0 0.2e0 0.1e0 0.1e0 0.2e0; 0.1e0 0.1e0 0.4e0 0.3e0 0.1e0 0.2e0; 0.1e0 0.1e0 0.1e0 0.2e0 0.1e0 0.1e0; 0.1e0 0.1e0 0.2e0 0.1e0 0.1e0 0.1e0]

discoFillRightToLeft = [0.4e0 0.2e0 0.1e0 0.1e0 0.3e0 0.2e0; 0.2e0 0.4e0 0.2e0 0.2e0 0.2e0 0.2e0; 0.1e0 0 0.4e0 0.1e0 0.1e0 0.2e0; 0 0.1e0 0.1e0 0.4e0 0.1e0 0.2e0; 0.2e0 0.1e0 0.1e0 0.1e0 0.2e0 0.1e0; 0.1e0 0.2e0 0.1e0 0.1e0 0.1e0 0.1e0]


discoFillInitial = [0.3e0,0.3e0,0.1e0,0.1e0,0.1e0,0.1e0]

discoFill = AugDblMatrix(discoFillLeftToRight, discoFillRightToLeft, discoFillInitial)

6×6 Matrix{Float64}:
 0.2  0.1  0.1  0.0  0.3  0.2
 0.4  0.2  0.0  0.3  0.3  0.2
 0.1  0.4  0.2  0.1  0.1  0.2
 0.1  0.1  0.4  0.3  0.1  0.2
 0.1  0.1  0.1  0.2  0.1  0.1
 0.1  0.1  0.2  0.1  0.1  0.1
6×6 Matrix{Float64}:
 0.4  0.2  0.1  0.1  0.3  0.2
 0.2  0.4  0.2  0.2  0.2  0.2
 0.1  0.0  0.4  0.1  0.1  0.2
 0.0  0.1  0.1  0.4  0.1  0.2
 0.2  0.1  0.1  0.1  0.2  0.1
 0.1  0.2  0.1  0.1  0.1  0.1
6-element Vector{Float64}:
 0.3
 0.3
 0.1
 0.1
 0.1
 0.1

In [113]:
dancedynMatrix = [
    0 0 0.2e0 0.2e0 0.1e0 0.1e0 0 0 0 0 0 0; 
    0 0 0.2e0 0.1e0 0.1e0 0.1e0 0 0 0 0 0 0; 
    0.2e0 0.2e0 0 0 0.2e0 0.1e0 0.1e0 0.1e0 0.1e0 0.2e0 0.1e0 0.2e0; 
    0.2e0 0.2e0 0 0 0.2e0 0.1e0 0.1e0 0.1e0 0.2e0 0.1e0 0.2e0 0.1e0; 
    0 0 0.1e0 0.2e0 0 0 0.2e0 0.1e0 0 0 0.1e0 0.2e0; 
    0 0 0.2e0 0.2e0 0 0 0.2e0 0.1e0 0 0 0.2e0 0.1e0; 
    0.1e0 0.2e0 0 0 0.1e0 0.2e0 0 0 0.2e0 0.2e0 0 0; 
    0.2e0 0.1e0 0 0 0.1e0 0.2e0 0 0 0.2e0 0.1e0 0 0; 
    0.1e0 0.2e0 0.1e0 0.2e0 0.1e0 0.1e0 0.1e0 0.2e0 0 0 0.2e0 0.2e0;
    0.2e0 0.1e0 0.2e0 0.1e0 0.1e0 0.1e0 0.1e0 0.2e0 0 0 0.2e0 0.2e0; 
    0 0 0 0 0 0 0.1e0 0.1e0 0.1e0 0.2e0 0 0; 
    0 0 0 0 0 0 0.1e0 0.1e0 0.2e0 0.2e0 0 0]


dancedynFinal = [
    #pp>, pp<, p>, p<, mp>, mp<, mf>, mf<, f>, f<, ff>, ff<
    zeros(1, 12); 
    zeros(1, 12); 
    0   0   0   0      0   0   0   0      0   0   0   0; 
    0   0   0   0      0.4 0.2 0.4 0.2    0.4 0.3 0.3 0.3; 
    zeros(1, 12); 
    zeros(1, 12);    
    zeros(1, 12); 
    zeros(1, 12); 
    0.2 0.3 0.2 0.3    0.2 0.2 0.2 0.2    0   0   0.2 0.2; 
    0.3 0.2 0.3 0.2    0.2 0.2 0.2 0.2    0   0   0.2 0.2; 
    0.2 0.3 0.2 0.3    0.1 0.2 0.1 0.2    0.2 0.4 0.1 0.2; 
    0.3 0.2 0.3 0.2    0.1 0.2 0.1 0.2    0.4 0.3 0.2 0.1]


dancedynInitial = [0,0,0.1e0,0.1e0,0.1e0,0.1e0,0.1e0,0.1e0,0.1e0,0.1e0,0.1,0.1]
#pp>, pp<, p>, p<, mp>, mp<, mf>, mf<, f>, f<, ff>, ff<

dancedyn = AugDblMatrix(dancedynMatrix, dancedynFinal, dancedynInitial)

12×12 Matrix{Float64}:
 0.0  0.0  0.2  0.2  0.1  0.1  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.2  0.1  0.1  0.1  0.0  0.0  0.0  0.0  0.0  0.0
 0.2  0.2  0.0  0.0  0.2  0.1  0.1  0.1  0.1  0.2  0.1  0.2
 0.2  0.2  0.0  0.0  0.2  0.1  0.1  0.1  0.2  0.1  0.2  0.1
 0.0  0.0  0.1  0.2  0.0  0.0  0.2  0.1  0.0  0.0  0.1  0.2
 0.0  0.0  0.2  0.2  0.0  0.0  0.2  0.1  0.0  0.0  0.2  0.1
 0.1  0.2  0.0  0.0  0.1  0.2  0.0  0.0  0.2  0.2  0.0  0.0
 0.2  0.1  0.0  0.0  0.1  0.2  0.0  0.0  0.2  0.1  0.0  0.0
 0.1  0.2  0.1  0.2  0.1  0.1  0.1  0.2  0.0  0.0  0.2  0.2
 0.2  0.1  0.2  0.1  0.1  0.1  0.1  0.2  0.0  0.0  0.2  0.2
 0.0  0.0  0.0  0.0  0.0  0.0  0.1  0.1  0.1  0.2  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.1  0.1  0.2  0.2  0.0  0.0
12×12 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.4  0.2  0.4  0.2  0.4  0.3  0.3

In [119]:
discoNoGhostsolo = abcsolo16patinst16ghost(4, discoNoGhostB, funkFill, dancedyn, discoPat, discoInst, "Disco 16th feel", variation="disco")
print(discoNoGhostsolo)

%%abc-include percussions-JBH.abh

I:linebreak $

#ifdef MIDI
#define !>! "[I:volinc 50]"
#define !Q! "[I:vol 20]" 
                %%%ghost
#define !U! "[I:vol 40]" 
                %%%unaccent
#define !N! "[I:volinc -25]" 
            %%%tenuto for trash hat
#else
#define !N! "\"^_\"" 
                %%%tenuto
#define !Q! !(.!!).! 
                %%%ghost
#define !U! !anti!  
                %%%unaccent
#endif

X:1
T:Computer-generated \`Etude for Solo Drum Kit
C:#abcsolo16patinst16ghost#29/disco
O:2021-12-31T17:52:28.381
M:4/4
L:1/8
Q:"Disco 16th feel" 1/4=80
K:none clef=perc
[V:1 clef=perc, stem=up]     % activate abc2xml.py map
%%voicemap drummap  % activate abcm2ps/abc2svg map
%%MIDI channel 10   % activate abc2midi map
%%MIDI program 0
%%flatbeams
%%propagate-accidentals not
%%pos ornament up
%%ornament up
%%MIDI fermatafixed
|:!mf!!>![^eF]/2!U![^e]/2!>![^e]/2!U![^e]/2 !>![^ecF]/2!U![^eF]/2!>![^e]/2!U![^eF]/2 !>![^eF]/2!U![^e]/2!>![^e]/2!U![^eF]/2 !>![^ecF]/2!U![^e]/2!>![^e]/2

In [120]:
#write a string to a file
open(f->print(f, discoNoGhostsolo), "disconoghosttest.abp", "w")

abptest("disconoghosttest.abp")

abcm2ps-8.14.12 (2021-07-14)
File disconoghosttest-ps.abc
File percussions-JBH.abh
Output written on disconoghosttest.xhtml (1 page, 1 title, 59540 bytes)
4.59 June 27 2021 abc2midi
writing MIDI file disconoghosttest.mid
Warning in line-char 41-0 : Track 1 Bar 19 has 9/2 units instead of 4 in repeat


disconoghosttest-ps.abc:29:47: warning: Line overfull (635pt of 628pt)


Process(`cmd /C start disconoghosttest.xhtml`, ProcessExited(0))

In [116]:
#0 = rest, 1 = bass, 2 = LH
                #00    #01  #02   #10  #11  #12  #zeros
discoBassMatrix = [0   0    0     0.2  1//3 0   0 0 0 #00
                    0  0    0     0.4  1//3 0.5    0 0 0  #01
                    0  0    0     0.4  1//3 0.5    0 0 0  #02
                    0.2 0.5  1//3   0    0    0   0 0 0  #10
                    0.4 0    1//3   0    0    0   0 0 0  #11
                    0.4 0.5  1//3   0    0    0   0 0 0    #12
                    zeros(3,9)]                 #zeros

sum(discoBassMatrix, dims=1)

discoBassInitial = [0, 0, 0, 1//3, 1//3, 1//3, 0, 0, 0]

discoBass = AugMatrix(discoBassMatrix, discoBassInitial)

9×9 Matrix{Float64}:
 0.0  0.0  0.0       0.2  0.333333  0.0  0.0  0.0  0.0
 0.0  0.0  0.0       0.4  0.333333  0.5  0.0  0.0  0.0
 0.0  0.0  0.0       0.4  0.333333  0.5  0.0  0.0  0.0
 0.2  0.5  0.333333  0.0  0.0       0.0  0.0  0.0  0.0
 0.4  0.0  0.333333  0.0  0.0       0.0  0.0  0.0  0.0
 0.4  0.5  0.333333  0.0  0.0       0.0  0.0  0.0  0.0
 0.0  0.0  0.0       0.0  0.0       0.0  0.0  0.0  0.0
 0.0  0.0  0.0       0.0  0.0       0.0  0.0  0.0  0.0
 0.0  0.0  0.0       0.0  0.0       0.0  0.0  0.0  0.0
9-element Vector{Rational{Int64}}:
 0//1
 0//1
 0//1
 1//3
 1//3
 1//3
 0//1
 0//1
 0//1

In [107]:
discosolo = abcsolo16patinst16ghost(4, discoBass, funkFill, dancedyn, discoPat, discoInst, "Disco 16th feel", variation="disco")
print(discosolo)

LoadError: MethodError: no method matching abctune16patinst16ghost(::Int64, ::Vector{Any}, ::Int64, ::Int64, ::Vector{Any}, ::Int64; variation="disco")
[0mClosest candidates are:
[0m  abctune16patinst16ghost(::Any, ::Any, ::Any, ::Any, ::Any, ::Any) at In[46]:2[91m got unsupported keyword argument "variation"[39m

In [153]:
writetune("discoghosttest.abp", discosolo)

abptest("discoghosttest.abp")

abcm2ps-8.14.12 (2021-07-14)
File discoghosttest-ps.abc
File percussions-JBH.abh
Output written on discoghosttest.xhtml (1 page, 1 title, 55575 bytes)
4.59 June 27 2021 abc2midi
writing MIDI file discoghosttest.mid
Warning in line-char 41-0 : Track 1 Bar 19 has 9/2 units instead of 4 in repeat


discoghosttest-ps.abc:29:44: warning: Line overfull (634pt of 628pt)


Process(`cmd /C start discoghosttest.xhtml`, ProcessExited(0))